In [104]:
# Install required packages (uncomment if needed)
# !pip install earthengine-api geemap

import ee
import geemap
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
import os

In [112]:
def download_landsat_rgb(image_id, out_name, region):
    """
    Downloads a Landsat image, selects RGB bands, clips it to a region, 
    visualises it in a notebook map, and exports it locally as a GeoTIFF.

    Parameters:
        image_id (str): Earth Engine Landsat image asset ID.
        out_name (str): Base filename for saving the output file.
        region (ee.Geometry): The region to clip and export.

    Returns:
        str: File path to the exported GeoTIFF.
    """

    image = ee.Image(image_id).select(rgb_bands).clip(region)
    
    # Display in notebook (optional)
    Map = geemap.Map()
    Map.centerObject(region, 10)
    Map.addLayer(image, vis_params, f'{out_name} RGB')
    Map.addLayer(region, {}, "Bounding Box")
    display(Map)
    
    # Export to local GeoTIFF
    export_path = os.path.join(output_dir, f'{out_name}.tif')
    geemap.ee_export_image(
        image,
        filename=export_path,
        region=region,
        scale=30,
        file_per_band=False
    )
    print(f"Image downloaded: {export_path}")
    return export_path

In [115]:
def reproject_raster(input_path, epsg=3413):
    """
    Reprojects a GeoTIFF raster to a specified EPSG coordinate system.

    Parameters:
        input_path (str): File path to the input raster.
        epsg (int): EPSG code for the target CRS (default = 3413 for NSIDC Polar Stereographic North).

    Returns:
        str: File path to the reprojected output raster.
    """
    filename = os.path.basename(input_path)
    output_path = os.path.join(output_dir, filename.replace('.tif', f'_EPSG{epsg}.tif'))
    target_crs = CRS.from_epsg(epsg)

    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=Resampling.nearest
                )
    
    print(f"Reprojection complete: {output_path}")
    return output_path

In [108]:
# Authenticate and initialise Earth Engine
ee.Authenticate()
ee.Initialize()

In [116]:
# Bounding box in WGS84 [xmin, ymin, xmax, ymax]
bbox = [-44.18926, 81.6518, -43.70383, 81.77820]
region = ee.Geometry.Rectangle(bbox)
# Output directory
repo_dir = os.getcwd()
output_dir = os.path.join(repo_dir, 'data')
os.makedirs(output_dir, exist_ok=True)

In [117]:
# Landsat image IDs and export names
image_info = [
    {
        "id": 'LANDSAT/LC08/C02/T1_L2/LC08_031247_20150801',
        "out_name": 'LC08_L2SP_031247_20150801_rgb_cropped'
    },
    {
        "id": 'LANDSAT/LC08/C02/T1_L2/LC08_030247_20140722',
        "out_name": 'LC08_L2SP_030247_20140722_rgb_cropped'
    }
]

# RGB bands (SR_B4 = red, SR_B3 = green, SR_B2 = blue)
rgb_bands = ['SR_B4', 'SR_B3', 'SR_B2']

# Visualization stretch for display
vis_params = {
    'bands': rgb_bands,
    'min': 5000,
    'max': 15000,
    'gamma': 1.4
}


In [118]:
for img in image_info:
    tif_path = download_landsat_rgb(img["id"], img["out_name"], region)
    reproject_raster(tif_path)

Map(center=[81.71491372706069, -43.94654500000049], controls=(WidgetControl(options=['position', 'transparent_…

Generating URL ...
Please wait ...
Data downloaded to /Users/jade/Documents/Lancaster PhD/Harder-Glacier-subglacial-flood-outburst/figures/data/LC08_L2SP_031247_20150801_rgb_cropped.tif
Image downloaded: /Users/jade/Documents/Lancaster PhD/Harder-Glacier-subglacial-flood-outburst/figures/data/LC08_L2SP_031247_20150801_rgb_cropped.tif
Reprojection complete: /Users/jade/Documents/Lancaster PhD/Harder-Glacier-subglacial-flood-outburst/figures/data/LC08_L2SP_031247_20150801_rgb_cropped_EPSG3413.tif


Map(center=[81.71491372706069, -43.94654500000049], controls=(WidgetControl(options=['position', 'transparent_…

Generating URL ...
Please wait ...
Data downloaded to /Users/jade/Documents/Lancaster PhD/Harder-Glacier-subglacial-flood-outburst/figures/data/LC08_L2SP_030247_20140722_rgb_cropped.tif
Image downloaded: /Users/jade/Documents/Lancaster PhD/Harder-Glacier-subglacial-flood-outburst/figures/data/LC08_L2SP_030247_20140722_rgb_cropped.tif
Reprojection complete: /Users/jade/Documents/Lancaster PhD/Harder-Glacier-subglacial-flood-outburst/figures/data/LC08_L2SP_030247_20140722_rgb_cropped_EPSG3413.tif
